## IMPORT LIBRARY

In [418]:
import pandas as pd
import numpy as np
from collections import Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

## COSINE SIMILIARITY

In [419]:
def Similarity(A,B) :
    splitA = A.split(' ')
    splitB = B.split(' ')
    CountA = Counter(splitA)
    CountB = Counter(splitB)
    
    intersection = set(CountA.keys()) & set(CountB.keys())
    pembilang = sum([CountA[x] * CountB[x] for x in intersection])

    sum1 = sum([CountA[x]**2 for x in CountA.keys()])
    sum2 = sum([CountB[x]**2 for x in CountB.keys()])
    pembagi = np.sqrt(sum1) * np.sqrt(sum2)
    if not pembagi:
        return 0.0
    else:
        return intersection,float(pembilang) / pembagi

## DATA PREPARATION

In [420]:
df = pd.read_csv('datasakit.csv')

In [421]:
df = df.rename(index=str, columns={"Tanda dan Gejala (Query)": "TnG", "Nama Penyakit (Data Diagnosis)": "penyakit"})

In [422]:
df = df.drop(columns=['No'])

In [423]:
df.head()

,TnG,penyakit
0,"menggigil, demam, sakit kepala",Malaria (bentuk benigma)
1,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid
2,"Mata lengket, mata berair, pandangan sedikit k...",NaN
3,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis
4,"Gusi bengkak, gusi kemerahan, gusi berdarah",NaN


In [424]:
data = df[df.isnull().any(axis=1)].reset_index()
data = data.drop(columns=['index'])

In [425]:
df = df.dropna().reset_index()
df = df.drop(columns=['index'])

In [426]:
df.head()

,TnG,penyakit
0,"menggigil, demam, sakit kepala",Malaria (bentuk benigma)
1,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid
2,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis
3,"Sakit gigi, gigi sensitif pada makanan dingin ...",Karies dentis
4,"Hidung tersumbat, bersin, batuk, sakit tenggor...",Common cold


In [427]:
data.head()

,TnG,penyakit
0,"Mata lengket, mata berair, pandangan sedikit k...",NaN
1,"Gusi bengkak, gusi kemerahan, gusi berdarah",NaN
2,"Batuk lebih dari tiga minggu, sesak napas atau...",NaN
3,"Demam, menggigil, suhu tubuh meningkat, batuk ...",NaN
4,"Demam, muntah, diare cair, ampas sedikit seper...",NaN


## DataFile Tokenization

In [398]:
tokenization1 = df["TnG"].str.split(', ') 
tokenization1.head()

0                     [menggigil, demam, sakit kepala]
1    [Kaku kuduk, penurunan kesadaran, muntah proye...
2    [Pipi bengkak, nyeri saat mengunyah, nyeri tes...
3    [Sakit gigi, gigi sensitif pada makanan dingin...
4    [Hidung tersumbat, bersin, batuk, sakit tenggo...
Name: TnG, dtype: object

## STEMMING KATA MENGGUNAKAN  NAZIEF ANDRIANI

In [399]:
for i in range(len(tokenization1)) :
    for j in range(len(tokenization1[i])) :
        tokenization1[i][j] = stemmer.stem(tokenization1[i][j])

In [400]:
for i in tokenization1 :
    print(i)

['gigil', 'demam', 'sakit kepala']
['kaku kuduk', 'turun sadar', 'muntah proyektil', 'sakit kepala']
['pipi bengkak', 'nyeri saat kunyah', 'nyeri testis']
['sakit gigi', 'gigi sensitif pada makan dingin atau panas', 'gusi bengkak']
['hidung sumbat', 'bersin', 'batuk', 'sakit tenggorok']
['hidung keluar lendir', 'choncae nasal biru', 'bersin']
['hidung lendir', 'demam', 'choncae nasal merah', 'bersin']
['hidung bau', 'nyeri di dalam hidung', 'ada krusta di dalam hidung']
['cuping telinga sakit', 'cuping telinga dungkul', 'demam']
['telinga sakit', 'keluar lendir dari telinga', 'dengar tidak ganggu', 'nyeri kunyah']
['nyeri telinga', 'telinga asa penuh', 'bisa demam']
['nyeri telinga', 'mukosa telinga edem', 'gendang telinga merah']
['nyeri telinga hebat', 'demam tinggi', 'gendang telinga tonjol bulging']
['nyeri telinga', 'keluar nanah dari telinga', 'demam kurang']
['sakit tenggorok', 'demam', 'faring warna merah']
['suara serak', 'demam', 'sakit kerongkongan']
['batuk', 'batuk darah',

## MENDAPATKAN FITUR DARI DATA

In [404]:
fitur = ['gigil']
for i in range(len(tokenization1)) :
    for j in range(len(tokenization1[i])) :
        check = False
        for k in range(len(fitur)) :
            intersec, Similiar = Similarity(tokenization1[i][j],fitur[k])
            if (Similiar > 0.8) :
                intersec = ' '.join(intersec)
                if (len(intersec) < len(fitur[k])):                    
                    fitur[k] = intersec
                check = True
                break
        if (not check) :
            fitur.append(tokenization1[i][j])
fitur.append('penyakit')

In [409]:
print(fitur[0:5])
print(len(fitur))

['gigil', 'demam', 'sakit kepala', 'kaku kuduk', 'turun sadar']
273


In [410]:
hasil = pd.DataFrame(data=0,columns=fitur,index=range(100))

In [414]:
hasil.head()

,gigil,demam,sakit kepala,kaku kuduk,turun sadar,muntah proyektil,pipi bengkak,kunyah nyeri,nyeri testis,sakit gigi,...,asa nyeri,jika tekan asa sakit,bengkak daun telinga jalar ke bagi belakang daun telinga sehingga sangat tonjol,inflamasi eksudatif dan gatal nyeri telinga,suara dengung,telinga sumbat,dengan tanda khas dapat hifa dan spora,sekret,sakit perut,penyakit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## DATA PENYAKIT DIPETAKAN PADA TABLE HASIL

In [428]:
for i in range(len(tokenization1)) :
    for j in range(len(tokenization1[i])) :
        if (tokenization1[i][j] in fitur) :
            hasil.loc[i,tokenization1[i][j]] = 1
            hasil.loc[i,'penyakit'] = df.loc[i,'penyakit']

In [431]:
hasil.head()

,gigil,demam,sakit kepala,kaku kuduk,turun sadar,muntah proyektil,pipi bengkak,kunyah nyeri,nyeri testis,sakit gigi,...,asa nyeri,jika tekan asa sakit,bengkak daun telinga jalar ke bagi belakang daun telinga sehingga sangat tonjol,inflamasi eksudatif dan gatal nyeri telinga,suara dengung,telinga sumbat,dengan tanda khas dapat hifa dan spora,sekret,sakit perut,penyakit
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Malaria (bentuk benigma)
1,0,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Meningitis + perdarahan subarachnoid
2,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,Parotitis
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Karies dentis
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Common cold


## DATA YANG AKAN DIDIAGNOSA PENYAKITNYA

In [382]:
data

,TnG,penyakit
0,"Mata lengket, mata berair, pandangan sedikit k...",NaN
1,"Gusi bengkak, gusi kemerahan, gusi berdarah",NaN
2,"Batuk lebih dari tiga minggu, sesak napas atau...",NaN
3,"Demam, menggigil, suhu tubuh meningkat, batuk ...",NaN
4,"Demam, muntah, diare cair, ampas sedikit seper...",NaN
5,"Nyeri kolik daerah pinggang, malaise, mual, ke...",NaN
6,ruam yang gatal terdri dari macula-makula keci...,NaN


## TOKENIZATION

In [435]:
tokenData = data["TnG"].str.split(', ') 

In [436]:
tokenData

0    [Mata lengket, mata berair, pandangan sedikit ...
1        [Gusi bengkak, gusi kemerahan, gusi berdarah]
2    [Batuk lebih dari tiga minggu, sesak napas ata...
3    [Demam, menggigil, suhu tubuh meningkat, batuk...
4    [Demam, muntah, diare cair, ampas sedikit sepe...
5    [Nyeri kolik daerah pinggang, malaise, mual, k...
6    [ruam yang gatal terdri dari macula-makula kec...
Name: TnG, dtype: object

## STEMMING

In [448]:
for i in range(len(tokenData)) :
    for j in range(len(tokenData[i])) :
        tokenData[i][j] = stemmer.stem(tokenData[i][j])

## PROSES DIAGONOSA PENYAKIT

In [449]:
for i in range(len(tokenData)) :
    fiturEveryData = []
    for j in range(len(tokenData[i])) :
        for k in fitur :
            intersec, similiar = Similarity(tokenData[i][j],k)
            if (similiar > 0.8) :
                fiturEveryData.append(k)
                break;
    pilih = [0,'']
    for j in range(len(hasil.index)) :
        cari = [0,'']
        for k in fiturEveryData :
            if (hasil.loc[j,k]==1) :
                cari[0] = cari[0] + 1
        if (pilih[0] < cari[0]) :
            pilih[0] = cari[0]
            pilih[1] = hasil.loc[j,'penyakit']
    if (pilih[0]!=0) :
        data.loc[i,'penyakit'] = pilih[1]

## HASIL

In [450]:
data

,TnG,penyakit
0,"Mata lengket, mata berair, pandangan sedikit k...",NaN
1,"Gusi bengkak, gusi kemerahan, gusi berdarah",Karies dentis
2,"Batuk lebih dari tiga minggu, sesak napas atau...",Shigolisis Anak
3,"Demam, menggigil, suhu tubuh meningkat, batuk ...",Malaria (bentuk benigma)
4,"Demam, muntah, diare cair, ampas sedikit seper...",Diare virus
5,"Nyeri kolik daerah pinggang, malaise, mual, ke...",Pelvic inflammatory disease
6,ruam yang gatal terdri dari macula-makula keci...,NaN
